
Reference:

https://www.kaggle.com/code/ammarnassanalhajali/k-fold-crossvalidation-coco-dataset-generator/notebook

https://www.kaggle.com/code/sagnik1511/uwmgit-data-preparation-from-scratch

In [ ]:
!pip install pycocotools

In [ ]:
!pwd

In [ ]:
import pycocotools
from pycocotools import mask
import json
import numpy as np
import pycocotools.mask as mask_util
from skimage import measure
import os
from tqdm import tqdm
from tqdm.notebook import tqdm
import cv2
import random
from itertools import groupby
import itertools
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from glob import glob
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib as mpl
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *

In [ ]:
!pwd

import the dataset to dataframe

In [ ]:
# creating a dataframe
# path ="/content/uw-madison-gi-tract-image-segmentation/"
# path ="../projet 8/"
path ="../input/uw-madison-gi-tract-image-segmentation/"

dataset_path = os.path.abspath(path) 
csv_path = os.path.abspath(path+"train.csv")
df = pd.read_csv(csv_path)

def fetch_file_from_id(root_dir, case_id):
    case_folder = case_id.split("_")[0]
    day_folder = "_".join(case_id.split("_")[:2])
    file_starter = "_".join(case_id.split("_")[2:])
    # fetching folder paths
    folder = os.path.join(root_dir, case_folder, day_folder, "scans")
    # fetching filenames with similar pattern
    file = glob(f"{folder}/{file_starter}*")[0]
    # returning the first file, though it will always hold one file.
    return file

# df["segmentation"] = df["segmentation"].astype("str")
df["case_id"] = df["id"].apply(lambda x: x.split("_")[0][4:])
df["day_id"] = df["id"].apply(lambda x: x.split("_")[1][3:])
df["slice_id"] = df["id"].apply(lambda x: x.split("_")[-1])
df["path"] = df["id"].apply(lambda x: fetch_file_from_id(path+'/train', x))
df['path']=df.path.astype('str')
df['case_id']=df.case_id.astype('str')
df['slice_id']=df.slice_id.astype('str')
df["filename"] = df["path"].apply(lambda x: x.split('/')[-1])
df['unique_filename']  = df.apply(lambda row: str(row.case_id) +'_'+ str(row.day_id)+'_'+str(row.filename), axis=1)
df["height"] = df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[2]).astype("int")
df["width"] = df["path"].apply(lambda x: os.path.split(x)[-1].split("_")[3]).astype("int")
df['size'] =df['height'].astype(str) +'x'+ df['width'].astype(str)

df.sample(5)


In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
(df.notna().mean(axis=0)*100).sort_values(ascending=True)

In [ ]:
taux = (df.notna().mean(axis=0)*100).sort_values(ascending=False)
plt.suptitle("Fill rate of variables in the dataset (%)")
taux.plot(kind='barh', figsize=(20, 15))

In [ ]:
df[df['segmentation'].isnull()].shape

In [ ]:
df[df['segmentation'].notnull()].shape

In [ ]:
df.dtypes

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
df_with_mask = df[df['segmentation'].notnull()] 
df_with_mask["segmentation"] = df["segmentation"].astype("str")
df_with_mask = df_with_mask.reset_index(drop=True)    
df_with_mask.head()

In [ ]:
df_with_mask.shape

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_with_mask['case_id'].value_counts().plot(kind='barh')
plt.title(str('repartition of case'),fontsize=15)

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_with_mask['day_id'].value_counts().plot(kind='barh')
plt.title(str('repartition of days traitment'),fontsize=15)

In [ ]:
fig = plt.figure(figsize=(15, 20))
df_with_mask['slice_id'].value_counts().plot(kind='barh')
plt.title(str('repartition of slice_id '),fontsize=15)

In [ ]:
pd.options.display.max_colwidth = 200
df_with_mask.path.head(1)[0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
fig = plt.figure(figsize=(5, 5))
img = mpimg.imread(df_with_mask.path[0])
imgplot = plt.imshow(img)
plt.show()

In [ ]:
df_with_mask.info()

In [ ]:
df_with_mask['size'].value_counts()

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10, 10))
ax = df_with_mask['size'].value_counts().plot(kind='bar', color=['r','b','g','y']) 

for p in ax.patches:
    ax.annotate(
                text=np.round(p.get_height(), decimals=2),
                xy=(p.get_x()+p.get_width()/2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 10),
                textcoords='offset points')
plt.title(str('rezolution images '),fontsize=15)
plt.show()

In [ ]:
df_with_mask.info()

In [ ]:
df_with_mask['class'].value_counts()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = df_with_mask['class'].value_counts().plot(kind='bar', color=['r','b','g']) 

for p in ax.patches:
    ax.annotate(text=np.round(p.get_height(), decimals=2),
                xy=(p.get_x()+p.get_width()/2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 10),
                textcoords='offset points')
plt.title(str('class annotation '),fontsize=15)
plt.show()

In [ ]:
df.columns

In [ ]:
fig = plt.figure(figsize=(20, 20))
df_with_mask['case_id'].value_counts().plot(kind='bar')

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

plt.rcParams['font.size'] = 10.0
mosaic(df_with_mask, ['class', 'size'], )
plt.figure(figsize=(40,40))

In [ ]:
plt.figure(figsize=(10,10))
ordre_quartier = df_with_mask.groupby('case_id')['class'].count().sort_values().index
sns.countplot(y="case_id",hue="class",data=df_with_mask, order=ordre_quartier)
plt.title(str('class annotation by case'),fontsize=15)

In [ ]:
df_with_mask.to_csv('/kaggle/working/madison.csv')

In [ ]:
pd.options.display.max_colwidth = 50
df_with_mask[df_with_mask['id']=='case121_day0_slice_0107']

In [ ]:
df_with_mask.shape

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

# From https://newbedev.com/encode-numpy-array-using-uncompressed-rle-for-coco-dataset
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle


# thanks to mohandass
# https://www.kaggle.com/code/mohanrobotics/cocoformat-dataset-creation-instance-segmentation
# i made some change to the id , the pycotools need numeric array id and not a string to work correctly
def create_coco_format_json(data_frame, classes, filepaths):
    images = []
    annotations = []
    categories = []
    count = 0
    
    # Additing categories
    for idx, class_ in enumerate(classes):
        categories.append(
            { 
                "id": idx,
                "name": class_
            }
        )
    i = 0
    for filepath in tqdm(filepaths):
        i += 1   
        image_id = i     
        file_id = ('_'.join((filepath.split("/")[-3] + "_" + filepath.split("/")[-1]).split("_")[:-4]))
        height_slice = int(filepath.split("/")[-1].split("_")[3])
        width_slice = int(filepath.split("/")[-1].split("_")[2])
        ids = data_frame.index[data_frame['id'] == file_id].tolist()
        # file_name = '/'.join(filepath.split("/")[2:])
        file_name = '/'.join(filepath.split("/")[3:])
        # print('/'.join(filepath.split("/")[2:]))
        if (len(ids) > 0):
            # Adding images which has annotations
            images.append(
                {
                    "id": image_id,
                    "file_id":file_id,#add id of the dataframe
                    "width":width_slice,
                    "height":height_slice,
                    "file_name": filepath
                }
            )
            for idx in ids:
                mk = rle_decode(data_frame.iloc[idx]['segmentation'], (height_slice, width_slice))
                ys, xs = np.where(mk)
                x1, x2 = min(xs), max(xs)
                y1, y2 = min(ys), max(ys)              
                contours,hierarchy = cv2.findContours(mk,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
                for id_, contour in enumerate(contours):
                    mask_image = np.zeros((mk.shape[0], mk.shape[1], 3),  np.uint8)
                    cv2.drawContours(mask_image, [contour], -1, (255,255,255), thickness=cv2.FILLED)
                    mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
                    mask_image_bool = np.array(mask_image, dtype=bool).astype(np.uint8)
                    ys, xs = np.where(mask_image_bool)
                    x1, x2 = min(xs), max(xs)
                    y1, y2 = min(ys), max(ys)
                    enc =binary_mask_to_rle(mask_image_bool)
                    seg = {
                        'segmentation':enc, 
                        'bbox': [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
                        'area': int(np.sum(mask_image_bool)),
                        'image_id':image_id, 
                        'category_id':classes.index(data_frame.iloc[idx]['class']), 
                        'iscrowd':0, 
                        'id': count
                    }
                    annotations.append(seg)
                    count +=1        
    # creating the dataset
    dataset_coco_format = {
        "categories": categories,
        "images": images,
        "annotations": annotations,
    }
    
    return dataset_coco_format

In [ ]:
# Creation of train test split
train_df, test_df = train_test_split(df_with_mask, test_size=0.2)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# print(train_df.head())
print("\n \nNumber of Train Images:{}".format(len(train_df)))
print("Number of Test Images:{}".format(len(test_df)))

In [ ]:
train_df.to_csv('./train_df.csv')
test_df.to_csv('./test_df.csv')

In [ ]:
indexes = test_df['id'].value_counts().sort_values(ascending=False).to_frame().iloc[:1000,:].index

In [ ]:
tmp_df = test_df[test_df['id'].isin(indexes)].sample(16)
tmp_df = tmp_df.reset_index(drop=True)
tmp_df.sort_values(['unique_filename']).sample(4)

In [ ]:
classes = ['small_bowel', 'large_bowel', 'stomach']
tmp_json = create_coco_format_json(tmp_df, classes, tmp_df['path'].tolist())

In [ ]:
# Code taken from: https://stackoverflow.com/a/65151218/12890869
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open('./tmp_json.json', 'w', encoding='utf-8') as f:
    json.dump(tmp_json, f, ensure_ascii=True, indent=4, default=np_encoder)        

In [ ]:
from pathlib import Path
annFile = Path(f'./tmp_json.json')
coco = COCO(annFile)
imgIds = coco.getImgIds()


imgs = coco.loadImgs(imgIds[0:2])
_,axs = plt.subplots(len(imgs),2,figsize=(40,15 * len(imgs)))
for img, ax in zip(imgs, axs):
    I = Image.fromarray(np.array(Image.open(img['file_name'])).astype("uint16"))
    annIds = coco.getAnnIds(imgIds=[img['id']])
    anns = coco.loadAnns(annIds)
    ax[0].imshow(I)
    ax[1].imshow(I)
    plt.sca(ax[1])
    coco.showAnns(anns, draw_bbox=True)

In [ ]:
import pandas as pd
info = coco.dataset

for k, v in info.items():
    if isinstance(v, int):
        info[k] = [v]
dataframe = pd.DataFrame.from_dict(info, orient='index')
dataframe

In [ ]:
classes = ['small_bowel', 'large_bowel', 'stomach']

train_json = create_coco_format_json(train_df, classes, train_df['path'].tolist())
test_json = create_coco_format_json(test_df, classes, test_df['path'].tolist())

In [ ]:
with open('./train_json.json', 'w', encoding='utf-8') as f:
    json.dump(train_json, f, ensure_ascii=True, indent=4, default=np_encoder)   
    
with open('./test_json.json', 'w', encoding='utf-8') as f:
    json.dump(test_json, f, ensure_ascii=True, indent=4, default=np_encoder)

In [ ]:
train_json = json.load(open('./train_json.json', 'r'))
test_json = json.load(open('./test_json.json', 'r'))

In [ ]:
IMAGE_TRAIN_DIR = './'
ANNOTATION_FILE_VAL = (IMAGE_TRAIN_DIR+'test_json.json')
ANNOTATION_FILE_TRAIN = (IMAGE_TRAIN_DIR+'train_json.json')

coco_train = COCO(ANNOTATION_FILE_TRAIN)
catIds_train = coco_train.getCatIds() 
imgIds_train = coco_train.getImgIds() 
imgDict_train = coco_train.loadImgs(imgIds_train) 
print(len(imgIds_train) , len(catIds_train))

coco_val = COCO(ANNOTATION_FILE_VAL)
catIds_val = coco_val.getCatIds()
imgIds_val = coco_val.getImgIds()
imgDict_val = coco_val.loadImgs(imgIds_val)
print(len(imgIds_val) , len(catIds_val))

In [ ]:
imgs = coco_train.loadImgs(imgIds_train[0:4])
_,axs = plt.subplots(len(imgs),2,figsize=(40,15 * len(imgs)))
for img, ax in zip(imgs, axs):
    print(img['file_id'])
    I = Image.open(img['file_name'])
    I = Image.fromarray(np.array(Image.open(img['file_name'])).astype("uint16"))
    annIds = coco_train.getAnnIds(imgIds=[img['id']])
    anns = coco_train.loadAnns(annIds)
    ax[0].imshow(I)
    ax[1].imshow(I)
    plt.sca(ax[1])
    coco_train.showAnns(anns, draw_bbox=True)